In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from keras.layers import LSTM, Dense, Dropout
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.models import load_model
from keras.layers import LSTM
from keras.utils.vis_utils import plot_model



## Data Wrangling

In [2]:
# loading
df = pd.read_csv('AAPL.csv')
labels = pd.DataFrame(df['Adj Close'])
features_titles = ['Open', 'High', 'Low', 'Volume']

# scaling
scaler = MinMaxScaler()
feature_transform = scaler.fit_transform(df[features_titles])
features= pd.DataFrame(columns=features_titles, data=feature_transform, index=df.index)

# splitting to training and testing set
timesplit = TimeSeriesSplit(n_splits=10)
for train_index, test_index in timesplit.split(feature_transform):
        x_train, x_test = feature_transform[:len(train_index)], feature_transform[len(train_index): (len(train_index)+len(test_index))]
        y_train, y_test = labels[:len(train_index)].values.ravel(), labels[len(train_index): (len(train_index)+len(test_index))].values.ravel()

# preprocessing for rnn
trainX = np.array(x_train)
testX = np.array(x_test)
x_train = trainX.reshape(x_train.shape[0], 1, x_train.shape[1])
x_test = testX.reshape(x_test.shape[0], 1, x_test.shape[1])

## Model Building

In [3]:
def lstm():
    model = Sequential()
    model.add(LSTM(32, input_shape=(1, trainX.shape[1]), 
        activation='relu', 
        return_sequences=False))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam') 
    return model

rnn = lstm()

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2021-12-27 21:48:37.987980: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-27 21:48:37.988359: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## Training

In [4]:
history = rnn.fit(x_train, y_train, epochs=100, batch_size=8, verbose=1, shuffle=False)

2021-12-27 21:48:39.647346: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/100


# Testing

In [ ]:
# prediction
y_pred = rnn.predict(x_test)

# predict vs true
plt.figure()
plt.plot(y_test, label='True Value')
plt.plot(y_pred, label='LSTM Value')
plt.title("Prediction by LSTM")
plt.xlabel('Time Scale')
plt.ylabel('Scaled USD')
plt.legend()
plt.show()